In [41]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from result_saver import SaverProvider, metadata_helper, metadata_loader
from soft_info.UnionFind import get_counts
import pandas as pd
import numpy as np

# Generate data

In [60]:
provider = SaverProvider()
backend = provider.get_backend('ibmq_manila')

In [64]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile

qc = QuantumCircuit(1, 1)
qc.x(0)
qc.measure(0, 0)
print(qc.draw(fold=-1))

initial_layout = [0]
transpiled_qc = transpile(qc, backend, initial_layout=initial_layout, optimization_level=0)

print(transpiled_qc.draw(fold=-1))

     ┌───┐┌─┐
  q: ┤ X ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0 
               ┌───┐┌─┐
      q_0 -> 0 ┤ X ├┤M├
               └───┘└╥┘
ancilla_0 -> 1 ──────╫─
                     ║ 
ancilla_1 -> 2 ──────╫─
                     ║ 
ancilla_2 -> 3 ──────╫─
                     ║ 
ancilla_3 -> 4 ──────╫─
                     ║ 
          c: 1/══════╩═
                     0 


In [65]:
job_metadata = metadata_helper(sampled_state=1, num_qubits=1)
job_id = backend.run(job_metadata, transpiled_qc, shots=1e6, meas_level=1, meas_return='single').job_id()

Custom Warning: updating metadata



# KDE Fitting

In [66]:
# load the job id that have 1e6 shots
job_metadata = metadata_loader().dropna(subset=['backend_options'])
job_metadata = job_metadata.join(job_metadata.backend_options.apply(pd.Series), rsuffix='_r')
job_metadata = job_metadata.drop(columns=['backend_options'])
filtered_data = job_metadata[job_metadata['shots'] == 1e6]
filtered_data

,notebook_name,job_id,job_name,job_metadata,tags,creation_date,day_str,time_str,backend_name,n_shots,...,num_qubits,test,descr,additional_metadata,meas_level_r,init_qubits,meas_return,skip_transpilation,shots,memory
23,KDE_Jak,cmvyazbk20pg008epbzg,NaN,{},[],2023-10-24 17:40:45.994179+02:00,24.10.23,17:40,ibmq_manila,NaN,...,NaN,NaN,NaN,"{'sampled_state': 1, 'num_qubits': 1}",1,True,single,True,1000000,False
22,KDE_Jak,cmvy9sf23ve0008zm1bg,NaN,{},[],2023-10-24 17:38:13.246957+02:00,24.10.23,17:38,ibmq_manila,NaN,...,NaN,NaN,NaN,"{'sampled_state': 1, 'num_qubits': 1}",1,True,single,True,1000000,False
21,KDE_Jak,cmvxwr3tjjb00081ccp0,NaN,{},[],2023-10-24 17:10:24.109707+02:00,24.10.23,17:10,ibmq_jakarta,NaN,...,NaN,NaN,NaN,"{'sampled_state': 0, 'num_qubits': 1}",1,True,single,True,1000000,False
20,KDE_Jak,cmvxnwzk20pg008epb80,NaN,{},[],2023-10-24 16:55:47.613514+02:00,24.10.23,16:55,ibmq_jakarta,NaN,...,NaN,NaN,NaN,"{'sampled_state': 1, 'num_qubits': 1}",1,True,single,True,1000000,False
19,plot_IQ_2,cmvwkbnjad30008ec01g,NaN,{},[],2023-10-24 15:42:07.022802+02:00,24.10.23,15:42,ibmq_manila,NaN,...,2.0,NaN,NaN,NaN,1,True,single,True,1000000,False
18,KDE_Jak,cmvvycsvpdgg008geya0,NaN,{},[],2023-10-24 14:57:23.519080+02:00,24.10.23,14:57,ibmq_jakarta,NaN,...,2.0,NaN,3 CNOTs,NaN,1,True,single,True,1000000,False


In [59]:
for job_id in filtered_data.job_id:
    job = provider.retrieve_job(job_id)

Custom Warning: Job ID cmvxwr3tjjb00081ccp0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider...

Custom Warning: Job ID cmvxwr3tjjb00081ccp0 is still running. Aborting...

Custom Warning: Job ID cmvxnwzk20pg008epb80 not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider...

Custom Warning: Job ID cmvxnwzk20pg008epb80 is still running. Aborting...

